In [15]:
from typing import NamedTuple

import kfp
from kfp.compiler import Compiler
from kfp.dsl import ContainerOp, pipeline
from kfp.components import create_component_from_func


In [16]:
def add_op(num1: int, num2: int) -> NamedTuple("Outputs", [("result", int)]):
    from utility import imported_func

    imported_func()

    return (num1 + num2,)

def substract_op(num1: int, num2: int) -> NamedTuple("Outputs", [("result", int)]):
    from utility import imported_func

    imported_func()
    
    return (num1 - num2,)

In [ ]:
@pipeline()
def pipeline_func(
    num1: int, # addition num
    num2: int, # addition num
    num3: int, # substract num
    docker_image_tag: str
):
    docker_image = f"asia-docker.pkg.dev/tk-test-data/kubebuild/workshop/train_model:{docker_image_tag}"
    
    op1 = create_component_from_func(add_op)(num1, num2)
    op1.container.image = docker_image
    result_op1 = op1.outputs["result"]

    op2 = create_component_from_func(substract_op)(result_op1, num3)
    op2.container.image = docker_image
    op2.after(op1)
    

In [ ]:
import os

Compiler().compile(
    pipeline_func=pipeline_func,
    package_path=os.path.join(".template", "pipeline.yml"),
)